# **Trabalho 2 - Memorial de Cálculo e Gráficos**

**Disciplina:** PPGEE0017 - Projeto de Circuitos Integrados - PPGEEC/UFBA

**Aluno:** André Paiva Conrado Rodrigues (N=2)

## 1 - Escopo do trabalho


### Especificações do circuito

Extração de características de amplificador OTA Miller com os seguintes parâmetros:

| **Parâmetro** | **Valor** | **Unidade**      |
|---------------|-----------|------------------|
| $W_{1,2}$     | 10        | $\mu \mathrm{m}$ |
| $L_{1,2}$     | 2         | $\mu \mathrm{m}$ |
| $W_{3,4}$     | 20        | $\mu \mathrm{m}$ |
| $L_{3,4}$     | 2         | $\mu \mathrm{m}$ |
| $W_5$         | 1         | $\mu \mathrm{m}$ |
| $L_5$         | 1         | $\mu \mathrm{m}$ |
| $W_6$         | 80        | $\mu \mathrm{m}$ |
| $L_6$         | 0,5       | $\mu \mathrm{m}$ |
| $W_7$         | 40        | $\mu \mathrm{m}$ |
| $L_7$         | 20        | $\mu \mathrm{m}$ |
| $C_L$         | 0,2       | pF               |
| $C_M$         | 2,0       | pF               |
| $V_{GG5}$     | -0,1      | V                |
| $V_{DD}$      | 0,6       | V                |
| $V_{SS}$      | -0,6      | V                |

### Parâmetros da Tecnologia

| **Símbolo** | **Definição**                  | **Unidade** | **Canal N**            | **Canal P**            |
|-------------|--------------------------------|-------------|------------------------|------------------------|
| $I_{SQ}$    | Corrente de norm. (quadrado)   | nA          | 353,38                 | 49,514                 |
| $n$         | Fator de rampa                 | -           | 1,3790                 | 1,2952                 |
| $\phi_{t}$  | Potencial Termodinâmico        | mV          | 26                     | 26                     |
| $V_{T0}$    | Tensão de limiar no equilíbrio | V           | 0,3618                 | -0,3554                |
| $U_{CRIT}$  | Campo Elétrico Crítico         | V/m         | $1,712 \cdot 10^{6}$   | $6,960 \cdot 10^{6}$   |
| $D_{SUB}$   | Parâmetro de modelo DIBL       | -           | $4,057 \cdot 10^{-6}$  | $3,781 \cdot 10^{-3}$  |
| $\eta_{0}$  | Parâmetro de modelo DIBL       | -           | $2,750 \cdot 10^{-6}$  | $1,151 \cdot 10^{-3}$  |
| $I_{t0}$    | Parâmetro de modelo DIBL       | m           | $3,1116 \cdot 10^{-8}$ | $3,1116 \cdot 10^{-8}$ |
| $\lambda$   | Parâmetro de modelo CLM        | -           | 1,9784                 | 0,8847                 |
| $DL$        | Encurtamento do canal          | nm          | 10,4                   | 14,8                   |
| $DW$        | Estreitamento do canal         | nm          | 0,24                   | 0                      |

In [1]:
# Importação de bibliotecas
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

In [2]:
# Especificações do amplificador OTA Miller
W_12 = 10e-6
L_12 = 2e-6
W_34 = 20e-6
L_34 = 2e-6
W_5 = 1e-6
L_5 = 1e-6
W_6 = 80e-6
L_6 = 0.5e-6
W_7 = 40e-6
L_7 = 20e-6
C_L = 0.2e-12
C_M = 2e-12
VGG5 = -0.1

In [3]:
# Parâmetros da tecnologia
Isq_p = 49.514e-9
Isq_n = 353.38e-9
N_p = 1.2952
N_n = 1.3790
VT0_p = -0.3554
VT0_n = 0.3618
Phi_t = 26e-3
Ucrit_p = 6.96e6
Ucrit_n = 1.712e6
Dsub_p = 3.781e-3
Dsub_n = 4.057e-6
Eta0_p = 1.151e-3
Eta0_p = 2.75e-6
It0 = 3.1116e-8
lambda_p = 0.8847
lambda_n = 1.9784
DL_p = 14.8
DL_n = 10.4
DW_p = 0
DW_n = 0.24

## 2 - Medição das correntes de dreno em simulação

In [4]:
# Correntes de dreno medidas em simulação
ID_M12 = 9.22461e-06 # Medido com sinal positivo
ID_M34 = 9.22461e-06 # Medido com sinal negativo
ID_M5 = 1.84492e-05 # Medido com sinal positivo
ID_M6 = 6.60956e-05 # Medido com sinal negativo
ID_M7 = 6.60956e-05 # Medido com sinal positivo

## 3 - Cálculo dos níveis de inversão

In [ ]:
# Função para cálculo de nível de inversão
def if_calc(Id, W, L, mos_type):
    if(mos_type == 'p' or mos_type == 'P'):
        return Id/(Isq_p*W/L)
    elif(mos_type == 'n' or mos_type == 'N'):
        return Id/(Isq_n*W/L)
    else:
        return -np.inf

In [6]:
If_M12 = if_calc(ID_M12, W_12, L_12, 'n')
If_M34 = if_calc(ID_M34, W_34, L_34, 'p')
If_M5 = if_calc(ID_M5, W_5, L_5, 'n')
If_M6 = if_calc(ID_M6, W_6, L_6, 'p')
If_M7 = if_calc(ID_M7, W_7, L_7, 'n')

print(f"If M12: {If_M12}")
print(f"If M34: {If_M34}")
print(f"If M5: {If_M5}")
print(f"If M6: {If_M6}")
print(f"If M7: {If_M7}")

If M12: 5.2207878204765406
If M34: 18.630306579957182
If M5: 52.2078216084668
If M6: 8.343044391485234
If M7: 93.5191578470768


|           | $I_{D} (\mu A)$ | $I_{f}$ |
|-----------|-----------------|---------|
| $M_{1,2}$ | 9,22461         | 5,221   |
| $M_{3,4}$ | 9,22461         | 18,630  |
| $M_5$     | 18,4492         | 52,208  |
| $M_6$     | 66,0956         | 8,343   |
| $M_7$     | 66,0956         | 93,519  |

## 4 - Cálculo dos parâmetros de desempenho

In [ ]:
# Funções auxiliares